# Código Auxiliar para a Segunda Avaliação.

In [ ]:
import pandas as pd
import sqlite3
from collections import defaultdict

As funções definidas abaixo, devem ser usada para acessar o banco de dados `bilheteria.db` e retornar os dados necessários para a resolução dos exercícios.


In [ ]:
def carrega_tabela(database, tabela):
    '''
    Carrega uma tabela de um banco de dados 
    SQLite em um DataFrame do Pandas.
    Parâmetros:
    database: str - Caminho para o arquivo do banco de dados SQLite.
    tabela: str - Nome da tabela a ser carregada.
    '''
    with sqlite3.connect(database) as conn:
        query = f"SELECT * FROM {tabela}"
        df = pd.read_sql_query(query, conn)
    return df

In [ ]:
def lista_tabelas(db_filename):
    '''
    Lista todas as tabelas em um banco de dados SQLite.
    Parâmetros:
    db_filename: str - Caminho para o arquivo do banco de dados SQLite.
    Retorna:
    DataFrame com os nomes das tabelas e suas contagens de linhas.
    '''
    # Connect to the SQLite database
    # conn = sqlite3.connect(db_filename)
    with sqlite3.connect(db_filename) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()

        table_row_counts = []
        for table in tables:
            table_name = table[0]
            query = f"SELECT COUNT(*) FROM {table_name};"
            cursor.execute(query)
            row_count = cursor.fetchone()[0]
            table_row_counts.append({"Table": table_name, "Row_Count": row_count})

        return pd.DataFrame(table_row_counts)

## Exemplos de uso das funções

In [ ]:
lista_tabelas('bilheteria.db')

# Utilice a celula abaixo para visualizar os dados

comente e descomente as tabelas que voce quer visualizar

In [51]:
# distribuidora = carrega_tabela("bilheteria.db", "distribuidora")
# filme = carrega_tabela("bilheteria.db", "filme")
# grupo_exibidor = carrega_tabela("bilheteria.db", "grupo_exibidor")
# exibidor = carrega_tabela("bilheteria.db", "exibidor")
# complexo = carrega_tabela("bilheteria.db", "complexo")
# sala = carrega_tabela("bilheteria.db", "sala")
# sessao = carrega_tabela('bilheteria.db', 'sessao')

complexo = carrega_tabela("bilheteria.db", "complexo")
sala = carrega_tabela("bilheteria.db", "sala")
sessao = carrega_tabela('bilheteria.db', 'sessao')
#sessao.data_exibicao = pd.to_datetime(sessao.data_exibicao, format='%d/%m/%Y')

# print(distribuidora)
# print(filme)
# print(filme.columns)
# print(grupo_exibidor)
# print(exibidor)
print(complexo)
print(sala)
print(sessao)


        id            municipio  UF  from_exibidor
0      438       BELO HORIZONTE  MG            437
1      895            SÃO PAULO  SP           1843
2      896          SANTO ANDRÉ  SP           1843
3      897            SÃO PAULO  SP           1843
4      898            SÃO PAULO  SP           1843
..     ...                  ...  ..            ...
678  55214        VÁRZEA GRANDE  MT           2379
679  55217  VALPARAÍSO DE GOIÁS  GO           2379
680  55428  SÃO JOSÉ DOS CAMPOS  SP          15263
681  55466             SÃO JOSÉ  SC           5935
682  55563       JOÃO MONLEVADE  MG          55562

[683 rows x 4 columns]
           id                                         nome  from_complexo
0     5000056  KINOPLEX IGUAÇU TOP SALA 01 - KINOEVOLUTION           2387
1     5000057                  KINOPLEX IGUAÇU TOP SALA 02           2387
2     5000058                  KINOPLEX IGUAÇU TOP SALA 03           2387
3     5000061                   SALA KINOPLEX MADUREIRA 01          

# Questao 1
1. Qual o total de bilheteria de todos os filmes, ou seja, o público que foi aos
filmes listados?

In [ ]:
def questao_1(database: str):
    sessao = carrega_tabela(database, "sessao")
    total_bilheteria_dataframe = sessao.publico
    total_bilheteria = sum(total_bilheteria_dataframe)
    print(f"O total de bilheteria dos filmes listados foi: {total_bilheteria}")
    return total_bilheteria

questao_1("bilheteria.db")

# Questao 2
2. Qual o filme de maior bilheteria em 2023, por país de origem?

In [82]:
def questao_2(database: str):
    #carregamos as tabelas necessarias com as funcoes
    filme = carrega_tabela(database, "filme") 
    sessao = carrega_tabela(database, "sessao")

    #a partir da tabela filme, seleccionamos as colunas id, titulo_original e pais_origem
    id_titulo_pais = filme[
        [
            "id",
            "titulo_original",
            "pais_origem"
        ]
    ]
    
    #a partir da tabela sessao, selecionamos as colunas filme_id e publico
    filme_id_publico = sessao[
        [
            "filme_id",
            "publico"
        ]
    ]

    #criamos um diccionarios
    bilheteria_de_cada_filme_por_id = defaultdict(int)    #chave: id de cada filme, valor: total de bilheteria 
    id_de_cada_filme_por_pais = defaultdict(list)         #chave: pais, valor: lista com os id's de cada filme desse pais
    nome_de_cada_filme_por_id = defaultdict(str)          #chave: id de cada filme, valor: nome de cada filme
    filme_de_maior_bilheteria_por_pais = defaultdict(str) #chave: pais, valor: filme com maior bilheteria desse pais

    #colocamos cada pais com seus filmes
    #colocamos cada filme_id com seu filme
    for index in id_titulo_pais.itertuples(index= False):
        filme_id = index.id
        titulo = index.titulo_original
        pais = index.pais_origem
        id_de_cada_filme_por_pais[pais].append(filme_id)
        nome_de_cada_filme_por_id[filme_id] = titulo
    
    #colocamos cada filme_id com seu publico total
    for index in filme_id_publico.itertuples(index=False):
        filme_id = index.filme_id
        publico = index.publico
        bilheteria_de_cada_filme_por_id[filme_id] += publico

    #colocamos cada pais com seu filme de maior bilheteria
    for cada_pais in id_de_cada_filme_por_pais.keys():
        bilheteria = 0
        filme = ""
        
        for cada_filme_id in id_de_cada_filme_por_pais[cada_pais]:
            if bilheteria_de_cada_filme_por_id[cada_filme_id] > bilheteria:
                bilheteria = bilheteria_de_cada_filme_por_id[cada_filme_id]
                filme = nome_de_cada_filme_por_id[cada_filme_id]

        filme_de_maior_bilheteria_por_pais[cada_pais] = filme

    filme_de_maior_bilheteria_por_pais = pd.DataFrame(list(filme_de_maior_bilheteria_por_pais.items()), columns=["pais","filme_de_maior_bilheteria"]).sort_values("pais").reset_index(drop=True)
    return filme_de_maior_bilheteria_por_pais

questao_2("bilheteria.db")

,pais,filme_de_maior_bilheteria
0,ALEMANHA,THE AMAZING MAURICE
1,ARGENTINA,BEM-VINDA VIOLETA
2,ARGÉLIA,LA DERNIÈRE REINE
3,AUSTRÁLIA,TALK TO ME
4,BELARUS (BIELORUSSIA),CATS IN THE MUSEUM
5,BRASIL,NOSSO SONHO
6,BÉLGICA,CLOSE
7,CANADÁ,PAW PATROL: THE MIGHTY MOVIE
8,CHILE,LA SOCIEDAD DE LA NIEVE
9,CHINA,OPERATION FORTUNE: RUSE DE GUERRE


# Questao 3
3. Crie um dataframe com as 100 cidades com maior bilheteria em 2023,
ordenadas de forma decrescente de bilheteria.

In [85]:
def questao_3(database: str):
    #precisamos de um diccionario com todas as cidades como chaves e uma lista com as id's das salas dessa cidade
    #depois um diccionario com a bilheteria total de cada sala
    #com isso criamos outro diccionarios onde calculamos a bilheteria total de cada cidade:chave somando a bilheteria:valor de todas as salas pertencentes a essa cidade
    #finalmente pegamos as 100 cidades com maior bilheteria em ordem decresente, pode ser em uma lista de tuplas 
    #convertemos essa lista de tuplas em um dataframe
    sala = carrega_tabela(database, "sala")
    complexo = carrega_tabela(database, "complexo")
    sessao = carrega_tabela(database, "sessao")
    
    sala_id_publico_total = sessao[["sala_id", "publico"]].groupby("sala_id").sum().reset_index()
    sala_id_compexo = sala[["id","from_complexo"]].rename(columns={"id":"sala_id"})
    complexo_e_cidade = complexo[["id","municipio"]].rename(columns={"id":"from_complexo"})

    sala_id_publico_e_complexo = sala_id_publico_total.merge(sala_id_compexo, how="left",on="sala_id")
    sala_id_publico_complexo_e_cidade = sala_id_publico_e_complexo.merge(complexo_e_cidade, how="left",on="from_complexo")

    cidades_bilheteria_total = sala_id_publico_complexo_e_cidade[["municipio","publico"]].groupby("municipio").sum(numeric_only=True)

    cem100_cidades_com_mais_bilheteria = cidades_bilheteria_total.sort_values("publico", ascending=False).head(100).reset_index().rename(columns={"publico": "bilheteria_total","municipio":"cidade"})
    
    return cem100_cidades_com_mais_bilheteria
    
questao_3("bilheteria.db")

,cidade,bilheteria_total
0,SÃO PAULO,13310470
1,RIO DE JANEIRO,8496306
2,BRASÍLIA,3272437
3,BELO HORIZONTE,3009414
4,CURITIBA,2965114
...,...,...
95,ARAÇATUBA,247992
96,VITÓRIA DA CONQUISTA,246490
97,PONTA GROSSA,241142
98,LAURO DE FREITAS,240784
